In [112]:
!pip install transformers torch

In [113]:
import pandas as pd
import spacy
nlp = spacy.load('en_core_web_sm')

In [114]:
import pickle
docs = pickle.load(open('raw_docs_filtered.pkl', 'rb'))

In [115]:
topics = {
    "credit": ["credit"],
    "performance": ["performance", "long term performance", "returns", "style drift", "systematic risks", "loss", "tail risks", "maximum drawdown", "track record", "long term"],
    "market": ["market"],
    "investment approach": ["investment approach", "sources of alpha", "investment process"],
    "portfolio": ["portfolio", "portfolio implementation", "risk management", "risk management system", "stop-loss limit", "liquidity", "leverage", "hedging", "risk"],
    "stress": ["stress", "performance short term", "stress test", "scenario analysis", "manager's outlook"],
    "team": ["team", "decision making", "staffing", "employees", "investment professionals", "investment teams", "incentive", "compensation", "investment manager"],
    "organization": ["organization", "esg policies", "administrator", "auditor", "prime broker", "trading system", "compliance", "IT infrastructure", "business continuity"],
    "fees": ["fees", "pass through fee", "management fee", "incentive fee"],
    "dealing terms": ["dealing terms", "lock-up", "pay-out ratio", "subscription", "redemption"],
    "asia": ["asia", "china", "csi"]
}


In [116]:
def extract_context(text, topics, window_size=5):
    """
    Extract context of given topics from text using token-based approach.
    
    Parameters:
    - text (str): The input text.
    - topics (dict): Dictionary with topics as keys and lists of keywords as values.
    - window_size (int): Number of tokens to include before and after the topic keyword.
    
    Returns:
    - dict: Dictionary with topics as keys and list of contexts as values.
    """
    # Process the text
    doc = nlp(text)
    tokens = [token.text for token in doc]
    topic_positions = {topic: set() for topic in topics}
    
    for topic, phrases in topics.items():
        for phrase in phrases:
            phrase_tokens = phrase.split()
            n = len(phrase_tokens)
            
            for i in range(len(tokens) - n + 1):
                ngram_tokens = tokens[i:i + n]
                
                if ngram_tokens == phrase_tokens:
                    start = max(0, i - window_size)
                    end = min(len(doc), i + window_size + 1)
                    context = tokens[start:end]
                    context_text = ' '.join(context)
                    topic_positions[topic].add(context_text)
                    
    
    return topic_positions

In [117]:
docs

,fund_mf_id,document_mf_id,fund_name,document_type,document_text
0,3253,123779,Candlestick Cayman Feeder Ltd,quarterly_report,Q1 2024\nC A N D L E S T I C K C A P I T A L\n...
1,2014,123730,Davidson Kempner Long-Term Distressed Opportun...,quarterly_report,confidential\nDanuta Neumann\nHedge Pole\nMay ...
2,3113,123731,Davidson Kempner Long-Term Distressed Opportun...,quarterly_report,confidential\nDanuta Neumann\nHedge Pole\nMay ...
3,936,123678,D.E. Shaw Valence International Fund LP,quarterly_report,1166 Avenue of the Americas\nNinth Floor\nNew ...
4,2780,123676,D.E. Shaw Composite International Fund,quarterly_report,SM\nCOMPOSITE\nQUARTERLY PERFORMANCE SUMMARY M...
...,...,...,...,...,...
902,4000,76164,Marshall Wace Funds PLC - MW Systematic Alpha ...,investor_letter,Marshall Wace Funds plc\n32 Molesworth Street\...
903,4011,100779,Marshall Wace UCITS Funds PLC - MW Systematic ...,investor_letter,LUMYNA-MARSHALL WACE UCITS SICAV\nSociété d’In...
904,4022,106079,Millburn Multi-Markets Ltd,investor_letter,Quant Millburn Hires Irina Bogacheva\nas Direc...
905,3380,106079,Millburn Resource Opportunities Fund Ltd,investor_letter,Quant Millburn Hires Irina Bogacheva\nas Direc...


In [118]:
def add_context(df):
    df['contexts'] = df['document_text'].apply(lambda x: extract_context(x, topics, window_size=5))
    return df

In [119]:
tmp = add_context(docs[:1])

C:\Users\viksu\AppData\Local\Temp\ipykernel_45724\2045900086.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['contexts'] = df['document_text'].apply(lambda x: extract_context(x, topics, window_size=5))


In [120]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from scipy.special import softmax

MODEL = f"soleimanian/financial-roberta-large-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)


def get_sentiments(contexts):
    df = pd.DataFrame(columns=['negative', 'neutral', 'positive'])
    for topic, context_list in contexts.items():
        if not context_list:
            continue
        context_list = [f"{context}" for context in context_list]
        encoded_input = tokenizer(context_list, return_tensors='pt', padding=True, truncation=True, max_length=512)
        output = model(**encoded_input)
        scores = [softmax(x.detach().numpy()) for x in output.logits]
        df2 = pd.DataFrame(scores, columns=['negative', 'neutral', 'positive'])
        df2['text'] = context_list
        df2['topic'] = topic
        df = pd.concat([df, df2], ignore_index=True)
    return df

C:\Users\viksu\Desktop\Diploma\TopicExtractor\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [121]:
sent = get_sentiments(tmp['contexts'][0])
sent.head()

C:\Users\viksu\AppData\Local\Temp\ipykernel_45724\1809251817.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df2], ignore_index=True)


,negative,neutral,positive,text,topic
0,0.000142,0.999623,0.000235,"- level , unaudited monthly performance inform...",performance
1,0.000229,0.999618,0.000153,"To the extent we quote returns on this index ,...",performance
2,0.000351,0.999497,0.000152,\n indication of the broader performance of th...,performance
3,0.000170,0.999533,0.000298,"As such , the net returns include “ new issue ”",performance
4,0.000790,0.998585,0.000625,offering \n documents . Net returns of other i...,performance


In [122]:
# find the average sentiment for each topic
def average_sentiments(df):
    del df['text']
    df = df.groupby('topic').mean()
    return df

In [123]:
average_sentiments(sent)

,negative,neutral,positive
topic,,,
fees,0.000253,0.901887,0.097861
market,0.142418,0.857353,0.000229
performance,0.102252,0.812978,0.084770
portfolio,0.168314,0.389982,0.441705
team,0.000215,0.999539,0.000246
